In [13]:
import  pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re 
import warnings
warnings.filterwarnings("ignore")


# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비
columns = ["검색어", "순번", "한자","품사","설명","링크"]
df = pd.DataFrame(columns=columns)
excutable_path = "../Library/chromedriver.exe"
driver = webdriver.Chrome(executable_path=excutable_path)
base_url = 'https://stdict.korean.go.kr'

def urlmake(links):
    source = links
    driver.get(source)
    req = driver.page_source
    soup = BeautifulSoup(req, "html.parser") 
    return soup

def listmake(word):
    num = isNone(word.select_one('.t_blue1 > sup'))
    china = isNone(word.select_one('dt> span'))
    poumsa =  isNone(word.select_one('dt'))
    content = isNone(word.select_one('font'))
    link = word.select_one('a').attrs['href']
    return num,china,poumsa,content,base_url +link

def isNone(word):
    if word != None and word.text != '':
        if len(word) > 3:
            index = startAndLast(re.sub('[\t\n\s]','',word.text))
            return re.sub('[\t\n\s]','',word.text)[index[0]:index[1]]
        return re.sub('[\t\n\s]','',word.text)
    else:
        return "Nope"

def startAndLast(word):
    ok = True
    start_index = 0
    last_index = 0
    for i in range(len(word)):
        if ok and word[i] == '「':
            start_index = i
        if word[i] == '」':
            last_index = i+1
            return start_index,last_index

        
        
        
        
search_data =  input("검색어를 입력해주세요.")


url = 'https://stdict.korean.go.kr/search/searchResult.do?pageSize=10&searchKeyword=' + str(search_data)

num = 0

search_soup = urlmake(url)

title = search_soup.select_one('.t_blue2')
if title != None:
    title = title.text.strip()
    words = search_soup.select('.result > li')
    for word in words:
        print(f'{columns[0]} : {title}')
        for i in range(len(listmake(word))):
            print(f'{columns[i+1]} : {listmake(word)[i]}') 
        print('----------------------------------------------------------------------------------------------')
        row = [title,*listmake(word)]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
else:
    text = search_soup.select_one('.no_result>li>div')
    if text == None:
        print(search_soup.select_one('.no_result').text)
    else:
        print(text.text)
        print('https://opendic.korean.go.kr/search/searchResult?query='+search_data)

검색어를 입력해주세요.학원
검색어 : 학원
순번 : 1
한자 : (學員)
품사 : 「명사」
설명 : 공부하는인원.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=362856&searchKeywordTo=3
----------------------------------------------------------------------------------------------
검색어 : 학원
순번 : 2
한자 : (學院)
품사 : 「명사」
설명 : 『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관.=학교.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=503671&searchKeywordTo=3
----------------------------------------------------------------------------------------------
검색어 : 학원
순번 : 3
한자 : (學園)
품사 : 「명사」
설명 : 『교육』학교및기타교육기관을통틀어이르는말.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=362857&searchKeywordTo=3
----------------------------------------------------------------------------------------------
검색어 : 학원
순번 : 4
한자 : (學園)
품사 : 「명사」
설명 : 『문학』1919년에발간된최초의학생잡지.창가,소설따위가발표되어소년ㆍ소녀들에게처음으로문학적인분위기에접하는계기를만들어주었다.
링크 : https://stdict.korean.go.kr/search/searchView.do?word_no=362858&searchKeywordTo=3
---------------------------------------

In [8]:
file = open('words.txt', 'rt', encoding ='utf8')
lines = file.readlines()
print(lines)

['학원\n', '학교\n', '역사\n', '수학\n', '소설']


In [9]:
f = open("words.txt", 'r', encoding = 'utf8')
lines = f.readlines()
for line in lines:
    line = line.strip()  
    print(line)
f.close()

학원
학교
역사
수학
소설


In [14]:
df

,검색어,순번,한자,품사,설명,링크
0,학원,1,(學員),「명사」,공부하는인원.,https://stdict.korean.go.kr/search/searchView....
1,학원,2,(學院),「명사」,『교육』일정한목적ㆍ교과과정ㆍ설비ㆍ제도및법규에의하여계속적으로학생에게교육을실시하는기관....,https://stdict.korean.go.kr/search/searchView....
2,학원,3,(學園),「명사」,『교육』학교및기타교육기관을통틀어이르는말.,https://stdict.korean.go.kr/search/searchView....
3,학원,4,(學園),「명사」,"『문학』1919년에발간된최초의학생잡지.창가,소설따위가발표되어소년ㆍ소녀들에게처음으로문...",https://stdict.korean.go.kr/search/searchView....


In [16]:
df.to_csv('학원.csv',index=False, encoding ='utf8') 

In [ ]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017')
db = client['test-database']
# db = client.test_database
collection = db['test-collection']
# collection = db.test-collection